In [13]:
from abenc_ph_mj18 import PH_ABE, mat_math
from charm.toolbox.pairinggroup import PairingGroup, ZR, G1, G2, GT, pair
from charm.toolbox.ABEnc import ABEnc

In [17]:
from abenc_ph_mj18 import Inner_Product

In [20]:
def ad_auth_setup(n, assump_size, ad, pks):

    pk_ad = []
    k = assump_size
    pk_ad = [[1] * k] * (k+1)

    for i in range(n-1):
        if i + 1 == ad:
            continue

        g_XTA = pks[str(i+1)]['g_1^{X^T A}']

        tmp = []
        for j in range(len(g_XTA)):
            tmp.append([x / y for x,y in zip(pk_ad[j], g_XTA[j])])
        
        pk_ad = tmp

    pk = pks[str(ad)]
    pk['g_1^{X^T A}'] = pk_ad
    
    pks[str(ad)] = pk

    return pks
    

In [21]:
def gen_omega(K, C, n, assump_size):
    H = K['g_2^h']
    k = assump_size
    C_i_s = C['C_i']

    C_ = [1] * (k + 1)
    #C_  = tmp
    for i in range(n-1):
        #print (len(C_), len(C_i_s[i]))
        C_ = [x / y for x,y in zip(C_, C_i_s[i])]

        #print (len(C_))

    return [pair(x,y) for x,y in zip(C_, H)]

In [38]:
n = 10
assump_size = 3
group = PairingGroup('MNT224')
math_lib = mat_math()

ph_abe = PH_ABE(n, assump_size, group, math_lib)

pp, msk = ph_abe.setup()
attributes = Inner_Product(group)
vec_x, vec_v = attributes.gen_x_v(n, assump_size)

pks, sks = ph_abe.auth_setup(pp)

In [40]:
ad = vec_v.index(0) + 1
ad_vec_v = [0] * (n-1) + [1]
print ('Authorized list: ', vec_v)
print ('Adversary:', ad, '\nVector belongs to AD:', ad_vec_v)
pks = ad_auth_setup(n, assump_size, ad, pks)

Authorized list:  [1, 0, 1, 0, 0, 1, 1, 0, 1, 1]
Adversary: 2 
Vector belongs to AD: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [41]:
M = group.random(GT)
print ('M: ', M)

C, vec_s = ph_abe.encrypt(pp, pks, vec_x, M)

M:  [[9179930058144160982031724517219801053539564500016631856097408525210, 1202828004069968285919388787173363650499937838762169761686762172258, 13023743049817406043053827756333361660438107038153046577477222526412], [11127999550334493716681488416220348734427419161140790601937153976550, 3823503143179356213115306727375863620355236173958378374157558499045, 4858313682959145238527509675204204981393664403313779788025434373780]]


In [42]:
K = ph_abe.keygen(pp, sks, vec_v)
print ('M_ (Usr):', ph_abe.decrypt(K, C, vec_v,pp))

M_ (Usr): [[9179930058144160982031724517219801053539564500016631856097408525210, 1202828004069968285919388787173363650499937838762169761686762172258, 13023743049817406043053827756333361660438107038153046577477222526412], [11127999550334493716681488416220348734427419161140790601937153976550, 3823503143179356213115306727375863620355236173958378374157558499045, 4858313682959145238527509675204204981393664403313779788025434373780]]


In [43]:
K_ = ph_abe.keygen(pp, sks, ad_vec_v)
omega = gen_omega(K_, C, n, assump_size)
M_ = ph_abe.decrypt(K_, C, ad_vec_v, pp)
print ('M_ (Adv):', M_ * math_lib.prod(omega))

M_ (Adv): [[9179930058144160982031724517219801053539564500016631856097408525210, 1202828004069968285919388787173363650499937838762169761686762172258, 13023743049817406043053827756333361660438107038153046577477222526412], [11127999550334493716681488416220348734427419161140790601937153976550, 3823503143179356213115306727375863620355236173958378374157558499045, 4858313682959145238527509675204204981393664403313779788025434373780]]
